In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Practise").getOrCreate()

In [2]:
df = spark.read.csv("test1.csv", header=True, inferSchema = True)

In [3]:
df.columns

['Name', 'age', 'Experience', 'Salary']

In [4]:
## in pyspark: we don't do train test split
## we will create the vectors of dependent and independent ones

from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ['age','Experience'], outputCol = 'variables')

In [5]:
output = featureassembler.transform(df)
output.show()

## we have created a vector for 2 independent features
## with the column name of independent features

+---------+---+----------+------+-----------+
|     Name|age|Experience|Salary|  variables|
+---------+---+----------+------+-----------+
|    Krish| 31|        10| 30000|[31.0,10.0]|
|Sudhanshu| 30|         8| 25000| [30.0,8.0]|
|    Sunny| 29|         4| 20000| [29.0,4.0]|
|     Paul| 24|         3| 20000| [24.0,3.0]|
|   Harsha| 21|         1| 15000| [21.0,1.0]|
|  Shubham| 23|         2| 18000| [23.0,2.0]|
+---------+---+----------+------+-----------+



In [6]:
df_final = output.select(['variables','Salary'])
df_final.show()

+-----------+------+
|  variables|Salary|
+-----------+------+
|[31.0,10.0]| 30000|
| [30.0,8.0]| 25000|
| [29.0,4.0]| 20000|
| [24.0,3.0]| 20000|
| [21.0,1.0]| 15000|
| [23.0,2.0]| 18000|
+-----------+------+



In [10]:
## Now tarin test split

from pyspark.ml.regression import LinearRegression

train_data, test_data = df_final.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='variables', labelCol='Salary')
regressor = regressor.fit(train_data)


In [12]:
test_data.show()

+-----------+------+
|  variables|Salary|
+-----------+------+
| [21.0,1.0]| 15000|
|[31.0,10.0]| 30000|
+-----------+------+



In [13]:
pred = regressor.evaluate(test_data)

In [15]:
pred.meanAbsoluteError

2440.8352668213374